## Notebook - 3 - SQL Server

## Topics Covered in this notebook

1. DML Triggers in SQL Server

## Triggers in SQL Server
In SQL server there are **3 types** of triggers
1. DML triggers
2. DDL triggers
3. Logon trigger

DML triggers are fired automatically in response to DML events ***(INSERT, UPDATE & DELETE)***

**DML triggers can be again classified into 2 types**
1. After triggers (Sometimes called as FOR triggers)
2. Instead of triggers

**After triggers, fires after the triggering action.** The INSERT, UPDATE, and DELETE statements, causes an after trigger to fire after the respective statements complete execution.

**INSTEAD of triggers, fires instead of the triggering action.** The INSERT, UPDATE, and DELETE statements, causes an INSTEAD OF trigger to fire INSTEAD OF the respective statement execution.

In [1]:
USE kudvenkatdb_master;

Commands completed successfully.

Total execution time: 00:00:00.014

In [ ]:
INSERT INTO tblEmployee VALUES ('Ben', 'Male',4800,'London')
DELETE FROM tblEmployee WHERE ID = 107;

In [11]:
CREATE table tblAudit(
    Id INT IDENTITY(1,1) PRIMARY KEY,
    AuditData NVARCHAR(255)
);


Commands completed successfully.

Total execution time: 00:00:00.002

In [14]:
CREATE TRIGGER tr_tblEmployee_ForInsert
ON tblEmployee
FOR INSERT
AS
BEGIN
    DECLARE @Id INT
    SELECT @Id = Id FROM inserted

    INSERT into tblAudit
    VALUES ('New Employee added with Id = '+CAST(@Id as nvarchar(5)) + ' at ' + CAST(GETDATE() as nvarchar(20)))
END

Commands completed successfully.

Total execution time: 00:00:00.003

In [2]:
CREATE TRIGGER tr_tblEmployee_ForDelete
ON tblEmployee
FOR DELETE
AS
BEGIN
    DECLARE @Id INT
    SELECT @Id = Id FROM deleted

    INSERT into tblAudit
    VALUES ('An existing employee deleted with Id = '+CAST(@Id as nvarchar(5)) + ' is deleted at ' + CAST(GETDATE() as nvarchar(20)))
END

Commands completed successfully.

Total execution time: 00:00:00.005

In [25]:
SELECT * FROM tblEmployee;
SELECT * FROM tblAudit;

(10 rows affected)

(9 rows affected)

Total execution time: 00:00:00.029

ID,Name,Gender,Salary,City
104,Todd,Male,2800,Sydney
102,John,Male,3500,London
100,Tom,Male,4000,London
103,Sam,Male,4500,London
110,Ben,Male,4800,London
105,Ben,Male,7000,New York
109,Russell,Male,8800,London
101,Pam,Female,3000,New York
108,Sara,Female,100000,London
106,Sneha,Female,100000,Sydney


Id,AuditData
1,New Employee added with Id = 110 at Jan 15 2024 12:50AM
2,An existing employee deleted with Id = 107 is deleted at Jan 15 2024 7:04AM
3,Employee with ID = 106 changed
4,Employee with ID = 106 changed
5,Employee with ID = 106 changed
6,Employee with ID = 106 changed Name from Sara Ali to Sara Ali Sara
7,Employee with ID = 106 changed
8,Employee with ID = 108 changed Name from James to SaraGender from Male to FemaleSalary from 6500 to 100000
9,Employee with ID = 106 changed Name from Sara Ali Sara to Sneha Salary from 4800 to 100000


## After Update Trigger
Note: The After trigger for UPDATE event, makes use of both ***inserted and deleted*** tables. The ***Inserted*** table contains the updated data and the ***deleted*** table contains the old data.

In [26]:
CREATE TRIGGER tr_tblEmployee_ForUpdate
on tblEmployee
for UPDATE
as
BEGIN   
    -- Logic
    -- Declare Fields
        DECLARE @Id INT
        
    
        DECLARE @OldName NVARCHAR(20), @NewName NVARCHAR(20)
        DECLARE @OldGender NVARCHAR(20), @NewGender NVARCHAR(20)
        DECLARE @OldSalary NVARCHAR(20), @NewSalary NVARCHAR(20)
        DECLARE @OldCity NVARCHAR(20), @NewCity NVARCHAR(20)

        DECLARE @AuditString NVARCHAR(1000)

    -- Store the updated data into temp table for creating the log
        SELECT * INTO #TempTable1 From inserted

    -- Iterate through each row
        WHILE (exists(SELECT @Id FROM #TempTable1))
        BEGIN
            SET @AuditString = ''

            SELECT TOP 1 @Id = ID, @NewName = Name,
            @NewGender = Gender, @NewSalary = Salary,
            @NewCity = City
            FROM #TempTable1

            SELECT @OldName = Name, @OldGender = Gender, 
            @OldSalary = Salary, @OldCity = City
            FROM deleted where ID = @Id

        -- Create the AUDITSTRING for the log...
            SET @AuditString = 'Employee with ID = ' + CAST(@Id as nvarchar(4)) + ' changed '

            if (@OldName <> @NewName)
                SET @AuditString = @AuditString + 'Name from ' + @OldName + ' to ' + @NewName

            if (@OldGender <> @NewGender)
                SET @AuditString = @AuditString + ' Gender from ' + @OldGender + ' to ' + @NewGender

            if (@OldSalary <> @NewSalary)
                SET @AuditString = @AuditString + ' Salary from ' + @OldSalary + ' to ' + @NewSalary

            if (@OldCity <> @NewCity)
                SET @AuditString = @AuditString + ' City from ' + @OldSalary + ' to ' + @NewSalary

        -- Insert into tblAudit
            INSERT INTO tblAudit VALUES(@AuditString)

        -- Delete the row from the tempTable, So next Iteration will proceed accordingly
            DELETE FROM #TempTable1 WHERE ID = @Id
        END
END

Commands completed successfully.

Total execution time: 00:00:00.005

In [ ]:
UPDATE tblEmployee SET NAME = 'Summit', Gender='Male', Salary= '100090' WHERE ID = 101

## Instead Of Triggers

In [36]:
select * from tblEmployee1;
select * from tblDepartment;

(10 rows affected)

(4 rows affected)

Total execution time: 00:00:00.015

ID,Name,Gender,Salary,DepartmentId
102,Mikey,Male,4000,1
103,Pam,Female,3000,3
104,John,Male,3500,1
105,Sam,Male,4500,2
106,Todd,Male,2800,2
107,Ben,Male,7000,1
108,Sara,Female,4800,3
110,James,Male,6500,NULL
111,Russell,Male,8800,NULL
112,Tom,Male,NULL,2


Id,DepartmentName,Location,DepartmentHead
1,HR,London,Rick
2,Payroll,Delhi,Ron
3,HR,New York,Christie
4,Other Department,Sydney,Cindrella


In [49]:
select * from vwEmployeesByDepartment;

(10 rows affected)

Total execution time: 00:00:00.016

Id,Name,Salary,Gender,DepartmentName
102,Mikey,4000,Male,HR
103,Pam,3000,Female,HR
104,John,3500,Male,HR
105,Sam,4500,Male,Payroll
106,Todd,2800,Male,Payroll
107,Ben,7000,Male,HR
108,Sara,4800,Female,HR
112,Tom,NULL,Male,Payroll
113,Sneha,100099,Female,HR
114,Pam Sara,10009999,Female,Other Department


In [46]:
Alter TRIGGER tr_vwEmployeesByDepartment_InsteadOfInsert
on vwEmployeesByDepartment
INSTEAD OF INSERT
as 
BEGIN
    -- Declare fields
        DECLARE @DeptId INT

    -- check if there is a valid DepartmentId for the given DepartmentName
        SELECT @DeptId = t.Id
        FROM tblDepartment t
        JOIN inserted
        ON inserted.DepartmentName = t.DepartmentName

    -- IF DepartmentID is null throw an error and stop processing
        if (@DeptId is null)
        BEGIN
            RAISERROR('Invalid Department Name. Statement terminated', 16, 1)
            RETURN
        END
    
    -- Finally insert into tblEmployee1 table
        INSERT INTO tblEmployee1(Name,Gender,Salary,DepartmentId)
        SELECT Name, Gender, Salary, @DeptId
        FROM inserted
END

Commands completed successfully.

Total execution time: 00:00:00.006

In [48]:
INSERT INTO vwEmployeesByDepartment (Name, Gender, Salary, DepartmentName) VALUES('Pam Sara','Female',10009999,'Other Department')

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.007